In [23]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
torch.cuda.is_available()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# print("当前设备名称：", torch.cuda.get_device_name(device) if device.type == 'cuda' else 'CPU')

In [24]:
def image2emb_naive(image, patch_size, weight):
    patch = F.unfold(image,kernel_size=patch_size,stride=patch_size).transpose(-1, -2)
    print(patch.shape)
    patch_embedding = patch @ weight
    return patch_embedding
# 通过卷积实现
def image2emb_conv(image, kernel, stride):
    # kernel训练中更新参数 
    # oc*ic*kh*kw
    conv_output = F.conv2d(image, kernel, stride=stride)
    ba, oc, oh, ow = conv_output.shape
    patch_embedding = conv_output.reshape((ba, oc, oh*ow)).transpose(-1,-2)
    return patch_embedding
# test code for image2emb
bs, ic, image_h, image_w = 1, 3, 8, 8
patch_size = 4
model_dim = 8
patch_depth = patch_size*patch_size*ic
image = torch.randn(bs, ic, image_h, image_w)
weight = torch.randn(patch_depth, model_dim)
patch_embedding_naive = image2emb_naive(image, patch_size, weight)
print(patch_embedding_naive.size())
print(patch_embedding_naive)
##################################
kernel = weight.transpose(0, 1).reshape((model_dim, ic, patch_size, patch_size))
patch_embedding_conv = image2emb_conv(image, kernel, patch_size)
print(patch_embedding_conv.size())
print(patch_embedding_conv)
torch.allclose(patch_embedding_naive, patch_embedding_conv)

torch.Size([1, 4, 48])
torch.Size([1, 4, 8])
tensor([[[-0.4444, -4.6532, -2.6227, -1.7000,  4.5345, -9.6457,  4.0828,
           0.3978],
         [ 8.5625,  5.2395,  5.9464,  2.4052,  7.3283,  5.2267,  5.6835,
           4.3923],
         [-2.9234, -0.9579,  3.0794, -1.8254, -5.7193,  5.1306,  6.5024,
          11.6948],
         [ 1.9704,  3.1219, -4.9349, -4.3023,  0.7116,  0.6098, 10.4769,
           0.2529]]])
torch.Size([1, 4, 8])
tensor([[[-0.4444, -4.6532, -2.6227, -1.7000,  4.5345, -9.6457,  4.0828,
           0.3978],
         [ 8.5625,  5.2395,  5.9464,  2.4052,  7.3283,  5.2267,  5.6835,
           4.3923],
         [-2.9234, -0.9579,  3.0794, -1.8254, -5.7193,  5.1306,  6.5024,
          11.6948],
         [ 1.9704,  3.1219, -4.9349, -4.3023,  0.7116,  0.6098, 10.4769,
           0.2529]]])


True

In [ ]:
# step2 prepend CLS token embedding